# Reacher Task, Deep RL for Continuous Control
---

### 1. Import the Necessary Packages

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
## Existing DDPG Code from Udacity DRLND repository
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from ddpg_agent import Agent

### 2. Instantiate the Environment and Agent

In [ ]:
env = UnityEnvironment(file_name='Reacher_Windows_x86_64/Reacher.exe')
agent = Agent(state_size=33, action_size=4, random_seed=5)


brain_name = env.brain_names[0]
brain = env.brains[brain_name]
START_DECAY = 250
GOAL_SCORE = 30

### 3. Train the Agent with DDPG

In [ ]:
def ddpg(n_episodes=600, max_t=300, print_every=100):
    SOLVED = False
    scores_deque = deque(maxlen=print_every)
    scores = []
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations
        agent.reset()
        score = 0
        decay = i_episode >= START_DECAY

        while True:
            action = agent.act(state, decay_noise=decay)
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations
            reward = env_info.rewards[0]
            done = env_info.local_done[0]
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_deque.append(score)
        scores.append(score)
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
        if np.mean(scores_deque) >= GOAL_SCORE and not SOLVED:
            SOLVED = True
            print(f'Environment solved! achieved an average score of 30 over 100 episodes at episode {i_episode}')
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            # agent kept improving after solving env, so left these at every 100 episodes. If performance degraded, could snapshot at solve by moving them up.
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
    return scores

scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
env.close()

### 4. Watch a Smart Agent!

In [ ]:
def run_agent(n_episodes=1, max_t=300, print_every=100):
    score = 0
    agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
    agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=False)[brain_name]
        state = env_info.vector_observations
        agent.reset()
        decay = i_episode >= START_DECAY

        while True:
            action = agent.act(state, add_noise=False, decay_noise=decay)
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations
            reward = env_info.rewards[0]
            done = env_info.local_done[0]
            # agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
    return score

score = run_agent()

print(f"Agent Complete Episode With a Score Of: {score}")